In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Image, Markdown, Code

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)
model = "openai/gpt-oss-20b:free"

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
def get_chat_completion(prompt):
    response = client.chat.completions.create(
    model = model,
    messages=[{'role':'user', "content":prompt}]
)
    return response.choices[0].message.content

In [ ]:
def get_chat_completion_from_messages(messages):
    response = client.chat.completions.create(
    model = model,
    messages=messages
)
    return response.choices[0].message.content

In [ ]:
print(get_chat_completion("States of India and thier Capitals"))

In [ ]:
react_demo_prompt = """Create an elegant, delightful React component for an Interview Feedback Form where:

1. The interviewer rates candidates across multiple dimensions using rubrics
2. Each rating must include specific evidence/examples
3. The final recommendation should auto-calculate based on a weighted scoring system
4. The UI should guide interviewers to give specific, behavioral feedback

The goal is to enforce structured, objective feedback gathering. A smart model should:
- Create a thoughtful rubric structure
- Add helpful prompts/placeholders
- Build in intelligent validation

Make sure to
 - Call the element FeedbackForm
 - Start the code with "use client"

Respond with the code only! Nothing else!"""

response2 = get_chat_completion(react_demo_prompt)

In [ ]:
display(Markdown(response2))

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user','content':f"{prompt}"})
    respons = get_chat_completion_from_messages(context)
    context.append({'role':'system','content':f"{respons}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.HTML(f"<div style='background-color:#F6F6F6;padding:8px'>{respons}</div>", width=600)))
    return pn.Column(*panels)

In [ ]:
import panel as pn
pn.extension()
panels = []
context = [{
    'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""
}]

inp = pn.widgets.TextInput(value='Hi',placeholder = 'Enter text here...')
button = pn.widgets.Button(name="Chat!")
interaction = pn.bind(collect_messages, button)
dashboard = pn.Column(
    inp,
    pn.Row(button),
    pn.panel(interaction, loading_indicator = True, height = 300)
)
pn.serve(dashboard, show=True)